# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read the cities_df.csv into a Dataframe
cities_df = pd.read_csv("output_data/cities_df.csv")
cities_df = cities_df.drop(["Unnamed: 0"], axis=1)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yeppoon,-23.1333,150.7333,80.60,34,40,9.22,AU,1619412131
1,bambous virieux,-20.3428,57.7575,78.01,88,75,14.97,MU,1619412131
2,hit,33.6416,42.8251,90.48,16,100,11.14,IQ,1619412131
3,kodiak,57.7900,-152.4072,42.80,81,1,9.22,US,1619412131
4,beringovskiy,63.0500,179.3167,31.55,90,44,2.10,RU,1619412131


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Create Location Dataframe by using "Lat" & "Lng"
lat_lngs_df = cities_df[["Lat","Lng"]].astype(float)

# Create List for the "Humidity"
humidity = cities_df["Humidity"].astype(float)
max_hum = humidity.max()

fig = gmaps.figure()
hum_layer = gmaps.heatmap_layer(lat_lngs_df, weights=humidity,
                                dissipating=False, max_intensity=max_hum,
                                point_radius=3)

fig.add_layer(hum_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Set criteria for vacation weather
vac_cities_df = cities_df.loc[(cities_df["Max Temp"]>70)&
                              (cities_df["Max Temp"]<80)&
                              (cities_df["Cloudiness"]==0)&
                              (cities_df["Wind Speed"]<10)].reset_index(drop=True)
len(vac_cities_df)

11

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Crreate Dataframe for hotel information
hotel_df = vac_cities_df.loc[:, ["Lat", "Lng", "City", "Country"]]
hotel_df

,Lat,Lng,City,Country
0,-29.4667,149.8500,moree,AU
1,-15.1961,12.1522,namibe,AO
2,-9.6658,-35.7353,maceio,BR
3,-25.1667,45.0500,beloha,MG
4,-20.3297,-40.2925,vila velha,BR
5,27.1810,31.1837,asyut,EG
6,-3.9014,-38.3911,aquiraz,BR
7,-34.5833,150.8552,flinders,AU
8,-35.3500,150.4667,ulladulla,AU
9,20.5169,-13.0499,atar,MR


In [6]:
# Test Call "moree" in "AU" to see the Structure.

# Geocoordinates
test_lat = -29.4667
test_lng = 149.8500
test_geo = f"{test_lat},{test_lng}"
test_r = 5000
test_type = "hotel"
test_keyword = "hotel"

# URL
test_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?\
location={test_geo}&radius={test_r}&type={test_type}&keyword={test_keyword}&\
key={g_key}"

response = requests.get(test_url).json()
data = response["results"]

# print(json.dumps(response, indent=2, sort_keys=True))
# print(data[0]["name"])

In [7]:
# Create Lists for Hotel Information
h_name =[]
h_address =[]
h_rating =[]
h_lat = []
h_lng = []

for index, row in hotel_df.iterrows():
    # Processing Info Reference
    cur_city = row["City"]
    cur_country = row["Country"]
    cur_lat = row["Lat"]
    cur_lng = row["Lng"]
    
    # Geocoordinates
    tgt_geo = f"{cur_lat},{cur_lng}"
    tgt_r = 5000
    tgt_type = "hotel"
    tgt_keyword = "hotel"
       
    # URL
    hs_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={tgt_geo}&radius={tgt_r}&type={tgt_type}&keyword={tgt_keyword}&key={g_key}"
    response = requests.get(hs_url).json()
    data = response["results"]
        
    try:
        h_name.append(data[0]["name"])
        h_address.append(data[0]["vicinity"])
        h_rating.append(data[0]["rating"])
        h_lat.append(data[0]["geometry"]["location"]["lat"])
        h_lng.append(data[0]["geometry"]["location"]["lng"])
        print(f"Processeing Record for a {tgt_keyword} in {cur_city}, {cur_country}")
        
    except(KeyError, IndexError):
        h_name.append(np.nan)
        h_address.append(np.nan)
        h_rating.append(np.nan)
        h_lat.append(np.nan)
        h_lng.append(np.nan)
        print(f"No Results/{tgt_keyword} found in {cur_city}, {cur_country}. Skipping...")

Processeing Record for a hotel in moree, AU
Processeing Record for a hotel in namibe, AO
Processeing Record for a hotel in maceio, BR
No Results/hotel found in beloha, MG. Skipping...
Processeing Record for a hotel in vila velha, BR
Processeing Record for a hotel in asyut, EG
Processeing Record for a hotel in aquiraz, BR
Processeing Record for a hotel in flinders, AU
Processeing Record for a hotel in ulladulla, AU
Processeing Record for a hotel in atar, MR
Processeing Record for a hotel in conceicao do araguaia, BR


In [8]:
# Combine extracted information to "hotel_df" and drop NaN
hotel_df["Hotel Name"] = h_name
hotel_df["Hotel Address"] = h_address
hotel_df["Hotel Rating"] = h_rating
hotel_df["Hotel Lat"] = h_lat
hotel_df["Hotel Lng"] = h_lng

hotel_df = hotel_df.dropna()
hotel_df

,Lat,Lng,City,Country,Hotel Name,Hotel Address,Hotel Rating,Hotel Lat,Hotel Lng
0,-29.4667,149.8500,moree,AU,Royal Hotel Moree,"54 Heber St, Moree",4.1,-29.463102,149.840634
1,-15.1961,12.1522,namibe,AO,Hotel Chik Chik Namibe,Namibe,3.6,-15.197832,12.157554
2,-9.6658,-35.7353,maceio,BR,Meridiano Hotel,"Av. Dr. Antônio Gouveia, 677 - Pajuçara, Maceió",4.6,-9.667967,-35.713902
4,-20.3297,-40.2925,vila velha,BR,Hotel Senac Ilha do Boi,"R. Bráulio Macedo, 417 - Ilha do Boi, Vitória",4.7,-20.310606,-40.282680
5,27.1810,31.1837,asyut,EG,Assiut hotels Armed Forces,الناصرية، مدينة الفتح،,3.9,27.202062,31.194197
6,-3.9014,-38.3911,aquiraz,BR,Lara hotel,"Rua Joana Maria da Conceição, 1734 - Prainha, ...",4.5,-3.895954,-38.353443
7,-34.5833,150.8552,flinders,AU,Lakeview Hotel Motel,"4 Government Rd, Oak Flats",3.8,-34.562229,150.830536
8,-35.3500,150.4667,ulladulla,AU,Ulladulla Harbour Motel,"29 Burrill St S, Ulladulla",4.0,-35.359545,150.476090
9,20.5169,-13.0499,atar,MR,Bab Sahara,"Unnamed Road, Atar",4.3,20.519051,-13.062103
10,-8.2578,-49.2647,conceicao do araguaia,BR,Tarumã Tropical Hotel,"Av. Brasília, 2120 - Capelinha, Conceição do A...",4.0,-8.272012,-49.260772


In [9]:
print(f"{len(vac_cities_df)} locations match vacation requirements. {len(hotel_df)} hotels found.")

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

11 locations match vacation requirements. 10 hotels found.


In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))